In [12]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# 괄호로 감싸진 숫자를 음수로 변환하는 함수
def convert_to_numeric(value):
    """괄호로 감싸진 숫자를 음수로 변환하는 함수"""
    if pd.isna(value):
        return value
    if isinstance(value, str):
        value_clean = value.strip()
        # 괄호로 감싸진 경우: (3000) -> -3000
        if value_clean.startswith('(') and value_clean.endswith(')'):
            value_clean = '-' + value_clean[1:-1]
        # 쉼표 제거
        value_clean = value_clean.replace(',', '').strip()
        return pd.to_numeric(value_clean, errors='coerce')
    return pd.to_numeric(value, errors='coerce')

# 1. CSV 파일 읽기
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
csv_path = os.path.join(project_root, 'data', '재무정보_final_1.csv')

if not os.path.exists(csv_path):
    csv_path = '/Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction/data/재무정보_final_1.csv'

df = pd.read_csv(csv_path, encoding='cp949')
data = df.copy()

# 컬럼명의 앞뒤 공백 제거
df.columns = df.columns.str.strip()

print("=== 원본 데이터 정보 ===")
print(f"데이터 shape: {df.shape}")
print(f"\n컬럼 목록:")
print(df.columns.tolist())
print(f"\nfield별 분포:")
print(df['field'].value_counts())
print(f"\n연도별 분포:")
print(df['연도'].value_counts().sort_index())

# 2. 숫자형 컬럼 선택 (결측치를 채울 컬럼들)
numeric_cols = ['매출액', '영업이익', '당기순이익', '자산', '부채', '자본', 
                '연구개발비', 'CAPEX', '유형자산_당기', '유형자산_전기', '유형자산_증감', '종업원_합계']

# 존재하는 컬럼만 선택
numeric_cols = [col for col in numeric_cols if col in df.columns]

# 숫자형 컬럼을 숫자형으로 변환 (문자열이나 object 타입인 경우)
# 괄호 처리: (3000) -> -3000, 쉼표와 공백 제거 후 변환
for col in numeric_cols:
    if df[col].dtype == 'object':
        # 문자열인 경우 괄호 처리, 쉼표와 공백 제거
        df[col] = df[col].apply(convert_to_numeric)
    else:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"\n=== 결측치 채울 컬럼 ({len(numeric_cols)}개) ===")
print(numeric_cols)

print(f"\n=== 결측치 현황 ===")
missing_info = df[numeric_cols].isnull().sum()
print(missing_info[missing_info > 0])

=== 원본 데이터 정보 ===
데이터 shape: (2766, 19)

컬럼 목록:
['기업명', '연도', 'bsnsr_reg_no', 'crno', 'dart_corp_code', 'sectors', 'field', '매출액', '영업이익', '당기순이익', '자산', '부채', '자본', '연구개발비', 'CAPEX', '유형자산_당기', '유형자산_전기', '유형자산_증감', '종업원_합계']

field별 분포:
field
부품    1302
장비    1068
소재     396
Name: count, dtype: int64

연도별 분포:
연도
2019    461
2020    461
2021    461
2022    461
2023    461
2024    461
Name: count, dtype: int64

=== 결측치 채울 컬럼 (12개) ===
['매출액', '영업이익', '당기순이익', '자산', '부채', '자본', '연구개발비', 'CAPEX', '유형자산_당기', '유형자산_전기', '유형자산_증감', '종업원_합계']

=== 결측치 현황 ===
매출액         792
영업이익        916
당기순이익       845
자산          780
부채          780
자본          794
연구개발비      1537
CAPEX      2278
유형자산_당기    1752
유형자산_전기    1779
유형자산_증감    1756
종업원_합계      913
dtype: int64


In [20]:
# 3. field별, 연도별로 상하위 10% 기업들의 평균으로 결측치 채우기
print(f"\n=== field별 상하위 10% 평균으로 결측치 채우기 시작 ===")

df_final = df.copy()

# df_final의 숫자형 컬럼도 숫자형으로 변환 (괄호 처리 포함)
for col in numeric_cols:
    if df_final[col].dtype == 'object':
        df_final[col] = df_final[col].apply(convert_to_numeric)
    else:
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

# 각 기업별로 처리
companies = df['기업명'].unique()
print(f"총 {len(companies)}개 기업 처리 중...")

for company in tqdm(companies, desc="기업 처리"):
    company_data = df[df['기업명'] == company].copy()
    company_field = company_data['field'].iloc[0] if company_data['field'].notna().any() else None
    
    if company_field is None:
        continue
    
    # 해당 기업의 각 연도별로 처리
    for target_year in range(2019, 2025):
        company_year_data = company_data[company_data['연도'] == target_year]
        
        if len(company_year_data) == 0:
            continue
        
        company_idx = company_year_data.index[0]
        
        # 각 숫자형 컬럼에 대해 처리
        for col in numeric_cols:
            # 이미 값이 있으면 스킵
            if pd.notna(df_final.loc[company_idx, col]):
                continue
            
            # 참조값 찾기 로직: 2019~2024년도 중 값이 있는 모든 연도들의 평균
            reference_value = None
            ref_values = []
            
            # 2019~2024년도 중 값이 있는 모든 연도 찾기
            for year in range(2019, 2025):
                year_data = company_data[company_data['연도'] == year]
                if len(year_data) > 0:
                    year_idx = year_data.index[0]
                    # 원본 데이터에서 찾기
                    ref_val = df.loc[year_idx, col]
                    # 괄호 처리 및 숫자형 변환
                    ref_val = convert_to_numeric(ref_val)
                    if pd.notna(ref_val) and np.isfinite(ref_val):
                        ref_values.append(ref_val)
            
            # 값이 있는 연도들의 평균을 참조값으로 사용
            if len(ref_values) > 0:
                reference_value = np.mean(ref_values)
            
            # 동일한 field의 다른 기업들 중에서
            # 해당 연도(target_year)에 값이 있는 기업들 찾기 (df_final에서 찾아야 이미 채워진 값도 반영)
            field_year_data = df_final[(df_final['field'] == company_field) & 
                                      (df_final['연도'] == target_year) & 
                                      (df_final[col].notna())].copy()
            
            # 자기 자신은 제외
            field_year_data = field_year_data[field_year_data['기업명'] != company]
            
            if len(field_year_data) == 0:
                continue
            
            # 참조값과의 차이 계산
            field_year_data[col] = pd.to_numeric(field_year_data[col], errors='coerce')
            field_year_data = field_year_data[field_year_data[col].notna()].copy()
            
            if len(field_year_data) == 0:
                continue
            
            # 참조값이 있는 경우: 참조값과 가까운 상하위 10% 선택
            if reference_value is not None:
                reference_value = pd.to_numeric(reference_value, errors='coerce')
                if pd.notna(reference_value):
                    field_year_data['diff'] = abs(field_year_data[col] - reference_value)
                    field_year_data = field_year_data.sort_values('diff')
                    
                    # 상하위 10% 선택 (최소 1개는 선택)
                    n_select = max(1, int(len(field_year_data) * 0.1))
                    if n_select == 0:
                        n_select = 1
                    
                    # 상하위 10% 기업들의 평균 계산
                    selected_data = field_year_data.head(n_select)
                    avg_value = selected_data[col].mean()
                else:
                    # 참조값이 유효하지 않으면 전체 평균 사용
                    avg_value = field_year_data[col].mean()
            else:
                # 참조값이 없으면 (모든 연도가 NaN인 경우) 해당 field의 해당 연도 전체 평균 사용
                avg_value = field_year_data[col].mean()
            
            # 평균값이 유효한 숫자인지 확인
            if pd.notna(avg_value) and np.isfinite(avg_value):
                # 평균값을 정수로 변환 (소수점 제거)
                avg_value_int = int(round(avg_value))
                # 결측치 채우기
                df_final.loc[company_idx, col] = avg_value_int

print(f"\n=== Imputation 완료 ===")
print(f"최종 데이터 shape: {df_final.shape}")


=== field별 상하위 10% 평균으로 결측치 채우기 시작 ===
총 461개 기업 처리 중...


기업 처리: 100%|██████████| 461/461 [00:14<00:00, 30.92it/s]


=== Imputation 완료 ===
최종 데이터 shape: (2766, 19)


In [21]:
# 4. 결과 확인
print(f"\n=== Imputation 후 결측치 현황 ===")
missing_after = df_final[numeric_cols].isnull().sum()
print(missing_after[missing_after > 0])
if missing_after.sum() == 0:
    print("모든 결측치가 채워졌습니다!")
else:
    print(f"\n남은 결측치: {missing_after.sum()}개")

# 샘플 데이터 확인
print(f"\n=== Imputation 결과 샘플 (첫 번째 기업) ===")
first_company = df_final['기업명'].iloc[0]
sample_data = df_final[df_final['기업명'] == first_company][['기업명', '연도', 'field'] + numeric_cols]
print(sample_data.head(10))


=== Imputation 후 결측치 현황 ===
Series([], dtype: int64)
모든 결측치가 채워졌습니다!

=== Imputation 결과 샘플 (첫 번째 기업) ===
        기업명    연도 field           매출액          영업이익         당기순이익  \
0  (주) 디이에스  2019    장비  1.979835e+10  4.194194e+09  3.573949e+09   
1  (주) 디이에스  2020    장비  1.931346e+10  3.592065e+09  3.726016e+09   
2  (주) 디이에스  2021    장비  1.928497e+10  4.198544e+09  3.277251e+09   
3  (주) 디이에스  2022    장비  2.027630e+10  4.122824e+09  3.336985e+09   
4  (주) 디이에스  2023    장비  1.877944e+10  4.302552e+09  3.483322e+09   
5  (주) 디이에스  2024    장비  1.975700e+10  4.378475e+09  3.747479e+09   

             자산            부채            자본        연구개발비         CAPEX  \
0  1.411165e+10  5.772188e+09  8.245511e+09  687090561.0  5.712662e+09   
1  1.452467e+10  5.548435e+09  8.381458e+09  699373773.0  1.489060e+10   
2  1.446257e+10  6.091180e+09  8.501727e+09  654463474.0  6.369178e+09   
3  1.360214e+10  5.861334e+09  8.160813e+09  682687638.0  9.940321e+09   
4  1.358662e+10  6.111614e+09  8.621966e

In [22]:
# 5. 결과 저장
output_path = os.path.join(project_root, 'data', '재무정보_final_imputed.csv')
df_final.to_csv(output_path, index=False, encoding='cp949')
print(f"\n=== 결과 저장 완료 ===")
print(f"저장 경로: {output_path}")
print(f"저장된 데이터 shape: {df_final.shape}")

# 데이터 요약 통계
print(f"\n=== Imputation 전후 비교 ===")
print("\n[Imputation 전]")
print(f"  총 결측치: {df[numeric_cols].isnull().sum().sum()}개")
print(f"  결측치 비율: {df[numeric_cols].isnull().sum().sum() / (len(df) * len(numeric_cols)) * 100:.2f}%")

print("\n[Imputation 후]")
print(f"  총 결측치: {df_final[numeric_cols].isnull().sum().sum()}개")
print(f"  결측치 비율: {df_final[numeric_cols].isnull().sum().sum() / (len(df_final) * len(numeric_cols)) * 100:.2f}%")


=== 결과 저장 완료 ===
저장 경로: /Users/roychoi/Documents/Github/sesac_project/Growth-Company-Prediction/data/재무정보_final_imputed.csv
저장된 데이터 shape: (2766, 19)

=== Imputation 전후 비교 ===

[Imputation 전]
  총 결측치: 14922개
  결측치 비율: 44.96%

[Imputation 후]
  총 결측치: 0개
  결측치 비율: 0.00%
